# Stream events from within tools

In [1]:
from langchain_core.callbacks import Callbacks
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool

from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

In [2]:
# define tool
@tool
async def get_items(
    place: str, callbacks: Callbacks
) -> str:  # <--- Accept callbacks (Python <= 3.10)
    """Use this tool to look up which items are in the given place."""
    template = ChatPromptTemplate.from_messages(
        [
            (
                "human",
                "Can you tell me what kind of items i might find in the following place: '{place}'. "
                "List at least 3 such items separating them by a comma. And include a brief description of each item..",
            )
        ]
    )
    chain = template | llm.with_config(
        {
            "run_name": "Get Items LLM",
            "tags": ["tool_llm"],
            "callbacks": callbacks,  # <-- Propagate callbacks (Python <= 3.10)
        }
    )
    chunks = [chunk async for chunk in chain.astream({"place": place})]
    return "".join(chunk.content for chunk in chunks)

In [3]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")
tools = [get_items]
agent = create_react_agent(llm, tools=tools)

In [4]:
async for event in agent.astream_events(
    {"messages": [("human", "what items are on the shelf?")]}, version="v2"
):
    tags = event.get("tags", [])
    if event["event"] == "on_chat_model_stream" and "tool_llm" in tags:
        print(event["data"]["chunk"].content, end="", flush=True)

/Users/aaron/Workspace/masterai/venv/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


1. Books: Books are typically found on shelves and can range from fiction novels to reference books. They are used for reading and gaining knowledge on various topics.

2. Picture frames: Picture frames are common decor items found on shelves to display photos or artwork. They come in various sizes and designs to complement the overall aesthetic of the space.

3. Decorative figurines: Decorative figurines are small statues or sculptures that are placed on shelves for aesthetic purposes. They can be made of various materials such as ceramic, metal, or wood, and can depict animals, people, or abstract shapes.